Web Server Case
===

Challenge from http://www.ashemery.com/dfir.html#Challenge1


Challenge questions from author:
---

To successfully solve this challenge, a report with answers to the tasks below is required:

1.    What type of attacks has been performed on the box?
1.    How many users has the attacker(s) added to the box, and how were they added?
1.    What leftovers (files, tools, info, etc) did the attacker(s) leave behind? (assume our team arrived in time and the attacker(s) couldn’t clean and cover their tracks)
1.    What software has been installed on the box, and were they installed by the attacker(s) or not?
1.    Using memory forensics, can you identify the type of shellcode used?
1.    What is the timeline analysis for all events that happened on the box?
1.    What is your hypothesis for the case, and what is your approach in solving it?
1.    Is there anything else you would like to add?

__Bonus Question__: what are the directories and files, that have been added by the attacker(s)? List all with proof.

__Important Note__: do not use commercial tools for your own learning benefit.



Memory Triage
===

Used these environment variables and _vol7.sh_ acript from dfirnotes.net rules [repo](https://github.com/dfirnotes/rules/).
```
VOLATILITY_FILEIN=memdump.mem
VOLATILITY_LOCATION=file:///cases/ashmery/ch1/memdump.mem
VOLATILITY_PROFILE=VistaSP2x86
OUT_FOLDER=.
```

Batch processed memdump.mem after identifing Volatility profile to try with _imageinfo_. Reviewed process tree and network connection data for suspicious process patterns. There connections stood out as possibly unusual:

```
...
0x3ef81c18         TCPv6    fe80::3816:d72e:759b:76b9:3306 ff02::1:3:51128      CLOSED           2804     mysqld.exe     
...
0x3f22c008         TCPv4    192.168.56.101:51157           192.168.56.1:5357    ESTABLISHED      1108     svchost.exe    
0x3ffc88f0         TCPv4    192.168.56.101:51160           192.168.56.1:139     CLOSED           4        System         
0x3ffd4008         TCPv4    192.168.56.101:51159           192.168.56.1:139     CLOSED           4        System        
```

In the process tree data, after the Xampp apache/mysql control panel, httpd, ftpd, and mysqld, we see two cmd.exe processes both with parent explorer.exe.

```
adric@marie:/cases/ashmery/ch1⟫ grep -B5 cmd *pstree.txt                                                       
. 0x83faa020:xampp-control.e                         2768    816      2    119 2015-08-23 10:32:17 UTC+0000
.. 0x83e4d7c0:httpd.exe                              2796   2768      1     92 2015-08-23 10:32:21 UTC+0000
... 0x83fd77a8:httpd.exe                             2880   2796    155    483 2015-08-23 10:32:26 UTC+0000
.. 0x83fd5200:FileZillaServer                        2856   2768      5     35 2015-08-23 10:32:25 UTC+0000
.. 0x83f9ec70:mysqld.exe                             2804   2768     23    570 2015-08-23 10:32:23 UTC+0000
. 0x83e7b7f8:cmd.exe                                  612    816      1     72 2015-08-23 10:30:44 UTC+0000
. 0x84259100:cmd.exe                                 1972    816      1     19 2015-09-02 09:28:30 UTC+0000
```

Parent process of XAmpp Control, VBoxTray, both cmds and the FTK Imager process that likey captured the memory is _explorer.exe_. This suggests these programs were all initiated by an interactive user.

```
adric@marie:/cases/ashmery/ch1⟫ grep 816 *pstree.txt*
 0x83e368e0:explorer.exe                              816    676     22    756 2015-08-23 10:30:34 UTC+0000
. 0x83e652a0:VBoxTray.exe                            1816    816      8    114 2015-08-23 10:30:38 UTC+0000
. 0x83f68300:FTK Imager.exe                          2120    816     13    382 2015-09-03 10:03:37 UTC+0000
. 0x83faa020:xampp-control.e                         2768    816      2    119 2015-08-23 10:32:17 UTC+0000
. 0x83e7b7f8:cmd.exe                                  612    816      1     72 2015-08-23 10:30:44 UTC+0000
. 0x84259100:cmd.exe                                 1972    816      1     19 2015-09-02 09:28:30 UTC+0000
```

The Volatility plugin _cmdline_ revealed no arguments for either _cmd.exe_ process (612 or 1972). 

```
adric@marie:/cases/ashmery/ch1⟫ vol.py cmdline -p 612
Volatility Foundation Volatility Framework 2.5
************************************************************************
cmd.exe pid:    612
Command line : "C:\Windows\System32\cmd.exe" 
adric@marie:/cases/ashmery/ch1⟫ vol.py cmdline -p 612 -p 1972
Volatility Foundation Volatility Framework 2.5
************************************************************************
cmd.exe pid:   1972
Command line : "C:\Windows\System32\cmd.exe" 
```

Digging further, I checked the batch processed output of the volatility _consoles_ plugin looking or command history or details from the cmd.exe processes in _memdump.mem-vol25c-consoles.txt_. There are some commands logged as well as some screen capture.

```
Cmd #0 at 0xe907c8: ipconfig
Cmd #1 at 0xe91af8: cls
Cmd #2 at 0xe91db0: ipconfig
Cmd #3 at 0x5a34bd0: net user user1 user1 /add
Cmd #4 at 0x5a34eb8: net user user1 root@psut /add
Cmd #5 at 0x5a34c10: net user user1 Root@psut /add
Cmd #6 at 0x5a24800: cls
Cmd #7 at 0x5a34c58: net /?
Cmd #8 at 0x5a34d88: net localgroup /?
Cmd #9 at 0x5a34f48: net localgroup "Remote Desktop Users" user1 /add
Cmd #10 at 0x5a34c70: net /?
Cmd #11 at 0xe911b0: netsh /?
Cmd #12 at 0xe907e8: netsh firewall /?
Cmd #13 at 0xe91218: netsh firewall set service type = remotedesktop /?
Cmd #14 at 0xe91288: netsh firewall set service type = remotedesktop enable
Cmd #15 at 0xe91300: netsh firewall set service type=remotedesktop mode=enable
Cmd #16 at 0xe91380: netsh firewall set service type=remotedesktop mode=enable scope=subnet
```

It certainly looks as though the cmd process "AttachedProcess: cmd.exe Pid: 612 Handle: 0x2d8" was used to add a user _user1_ (at least) and grant the user remote access via group membership, and that firewall rules were changed to allow Remote Desktop access to the system. We'll need to dig deeper and probably look at the disk evidence to find out how that occurred. Also, there were mistyped commands and usuage statements displayed, storngly suggesting an interactive human user was responsible.

A ClamAV scan of the modules dumped by the batch Voaltility run found no known malware.

Disk Ingest
===

After extraction we have a raw disk image. We can check the layout with Sleuthkit _mmls_ and then mount it up before processing the raw image into Plaso for super timeline.

```
$ file s4a-challenge4
s4a-challenge4: x86 boot sector

adric@marie:/cases/ashmery/ch1⟫ mmls s4a-challenge4
DOS Partition Table
Offset Sector: 0
Units are in 512-byte sectors

      Slot      Start        End          Length       Description
000:  Meta      0000000000   0000000000   0000000001   Primary Table (#0)
001:  -------   0000000000   0000002047   0000002048   Unallocated
002:  000:000   0000002048   0052426751   0052424704   NTFS / exFAT (0x07)
003:  -------   0052426752   0052428799   0000002048   Unallocated
```

Mount NFTS with _mountwin_ alias options from SIFT and calculated offset from partition table in mmls output.

```
sudo mount -o ro,loop,show_sys_files,streams_interface=windows,offset=$((2048 * 512)) s4a-challenge4 /mnt/windows_mount
```

Ingest entire disk "kitchen sink" style with simple Plaso invocation:

```
log2timeline.py ch1.plaso s4a-challenge4
```

Plaso info:

```
************************** Plaso Storage Information ***************************
            Filename : ch1.plaso
      Format version : 20160715
Serialization format : json
--------------------------------------------------------------------------------

*********************************** Sessions ***********************************
ef3aea58-1fec-47cd-b2a7-4ea67949e84d : 2017-02-05T06:14:17.586957+00:00
--------------------------------------------------------------------------------

**************** Session: ef3aea58-1fec-47cd-b2a7-4ea67949e84d *****************
                Start time : 2017-02-05T06:14:17.586957+00:00
           Completion time : 2017-02-05T06:42:22.275277+00:00
              Product name : plaso
           Product version : 1.5.1
    Command line arguments : /usr/bin/log2timeline.py ch1.plaso s4a-challenge4
  Parser filter expression : win7
Enabled parser and plugins : android_app_usage, asl_log, bencode,
                             bencode/bencode_transmission,
                             bencode/bencode_utorrent, binary_cookies, bsm_log,
                             chrome_cache, chrome_preferences, cups_ipp,
                             custom_destinations, dockerjson, dpkg, esedb,
                             esedb/esedb_file_history, esedb/msie_webcache,
                             filestat, firefox_cache, firefox_cache2, hachoir,
                             java_idx, lnk, mac_appfirewall_log, mac_keychain,
                             mac_securityd, mactime, macwifi,
                             mcafee_protection, mft, msiecf, olecf,
                             olecf/olecf_automatic_destinations,
                             olecf/olecf_default, olecf/olecf_document_summary,
                             olecf/olecf_summary, openxml, opera_global,
                             opera_typed_history, pe, plist, plist/airport,
                             plist/apple_id, plist/ipod_device,
                             plist/macosx_bluetooth,
                             plist/macosx_install_history, plist/macuser,
                             plist/maxos_software_update, plist/plist_default,
                             plist/safari_history, plist/spotlight,
                             plist/spotlight_volume, plist/time_machine,
                             pls_recall, popularity_contest, prefetch,
                             recycle_bin, recycle_bin_info2, rplog, sccm,
                             selinux, skydrive_log, skydrive_log_old, sqlite,
                             sqlite/android_calls, sqlite/android_sms,
                             sqlite/appusage, sqlite/chrome_cookies,
                             sqlite/chrome_extension_activity,
                             sqlite/chrome_history, sqlite/firefox_cookies,
                             sqlite/firefox_downloads, sqlite/firefox_history,
                             sqlite/google_drive, sqlite/imessage,
                             sqlite/kik_messenger, sqlite/ls_quarantine,
                             sqlite/mac_document_versions,
                             sqlite/mackeeper_cache, sqlite/skype,
                             sqlite/twitter_ios, sqlite/zeitgeist,
                             symantec_scanlog, syslog, syslog/cron, syslog/ssh,
                             usnjrnl, utmp, utmpx, winevt, winevtx,
                             winfirewall, winiis, winjob, winreg,
                             winreg/appcompatcache, winreg/bagmru,
                             winreg/ccleaner, winreg/explorer_mountpoints2,
                            winreg/explorer_programscache,
                             winreg/microsoft_office_mru,
                             winreg/microsoft_outlook_mru,
                             winreg/mrulist_shell_item_list,
                             winreg/mrulist_string,
                             winreg/mrulistex_shell_item_list,
                             winreg/mrulistex_string,
                             winreg/mrulistex_string_and_shell_item,
                             winreg/mrulistex_string_and_shell_item_list,
                             winreg/msie_zone, winreg/mstsc_rdp,
                             winreg/mstsc_rdp_mru, winreg/network_drives,
                             winreg/userassist, winreg/windows_boot_execute,
                             winreg/windows_boot_verify, winreg/windows_run,
                             winreg/windows_sam_users, winreg/windows_services,
                             winreg/windows_shutdown,
                             winreg/windows_task_cache,
                             winreg/windows_timezone,
                             winreg/windows_typed_urls,
                             winreg/windows_usb_devices,
                             winreg/windows_usbstor_devices,
                             winreg/windows_version, winreg/winlogon,
                             winreg/winrar_mru, winreg/winreg_default,
                             xchatlog, xchatscrollback, zsh_extended_history
        Preferred encoding : UTF-8
                Debug mode : False
               Filter file : N/A
         Filter expression : N/A
--------------------------------------------------------------------------------
************************* Events generated per parser **************************
                Parser (plugin) name : Number of events
--------------------------------------------------------------------------------
                      appcompatcache : 360
                              bagmru : 46
               explorer_mountpoints2 : 38
              explorer_programscache : 2
                            filestat : 262786
                                 lnk : 320
                      mrulist_string : 3
           mrulistex_shell_item_list : 3
                    mrulistex_string : 1
mrulistex_string_and_shell_item_list : 1
                           msie_zone : 66
                              msiecf : 104
                       olecf_default : 11
                                  pe : 53511
                         shell_items : 690
                          userassist : 22
                windows_boot_execute : 14
                         windows_run : 3
                    windows_services : 2257
                    windows_shutdown : 6
                  windows_task_cache : 92
                    windows_timezone : 7
                  windows_typed_urls : 1
                 windows_usb_devices : 18
                    windows_timezone : 7
                  windows_typed_urls : 1
                 windows_usb_devices : 18
                     windows_version : 6
                             winevtx : 3455
                            winlogon : 9
                      winreg_default : 379419
                               Total : 703251
--------------------------------------------------------------------------------

*********************************** Error: 0 ***********************************
           Message : unable to open file with error:
                     pymsiecf_file_open_file_object: unable to open file.
                     libmsiecf_hash_table_read: unsupported signature.
                     libmsiecf_io_handle_read_hash_table: unable to read hash
                     table at offset: 16384. libmsiecf_file_open_read: unable
                     to read hash table. libmsiecf_file_open_file_io_handle:
                     unable to read from file handle.
      Parser chain : msiecf
Path specification : type: OS, location: /cases/ashmery/ch1/s4a-challenge4
                   : type: TSK_PARTITION, location: /p1, part index: 2, start
                     offset: 0x00100000
                   : type: TSK, inode: 62329, location:
                     /Users/Administrator/AppData/Local/Microsoft/Windows/History/History.IE5/MSHist012015090320150904/index.dat
--------------------------------------------------------------------------------
No analysis reports stored.
```

With disk now mounted we can extract key files for more processing and extract supertimeline data for analysis.

```
adric@marie:/cases/ashmery/ch1⟫ psort.py ch1.plaso -o l2tcsv -w ch1_supertl_ks.csv

plaso - psort version 1.5.1

Storage file    : ch1.plaso

Identifier              PID     Status          Events          Tags            Reports
Main                    21474   exporting       703251 (0)      0 (0)           0 (0)

Processing completed.

*********************************** Counter ************************************
        Events processed : 703251
Duplicate events removed : 22562
--------------------------------------------------------------------------------
adric@marie:/cases/ashmery/ch1⟫ wc -l ch1_supertl_ks.csv 
680690 ch1_supertl_ks.csv
```

Given the role of the system the web logs should be of interest. Once we link the web logs with the process creation we should have some times worth investigating in the super timeline.

XAMPP Web Logs
===

The Apache logs are found in the mounted disk image in an obvious location in the C:\xampp\ folder structure.

```
adric@marie:/cases/ashmery/ch1⟫ ls -lh /mnt/windows_mount/xampp/apache/logs/
total 1.7M
-rwxrwxrwx 1 root root 1.6M Sep  3  2015 access.log
-rwxrwxrwx 1 root root 5.4K Aug 23  2015 error.log
-rwxrwxrwx 1 root root    6 Aug 23  2015 httpd.pid
-rwxrwxrwx 1 root root 3.1K Jul 12  2015 install.log
-rwxrwxrwx 2 root root    0 Aug 23  2015 ssl_request.log
```

A quick peak at the access logs' last two lines to see what the fields and time format look like is quite suggestive of our scenario. We also see that Apache is logging in UTC-7 so we'll need to be mindful of that comparing to our other time data:

```
adric@marie:/mnt/windows_mount/xampp/apache/logs⟫ tail -2 access.log 
192.168.56.102 - - [03/Sep/2015:00:31:30 -0700] "POST /dvwa/vulnerabilities/upload/ HTTP/1.1" 200 4722 "http://192.168.56.101/dvwa/vulnerabilities/upload/" "Mozilla/5.0 (X11; Linux x86_64; rv:38.0) Gecko/20100101 Firefox/38.0 Iceweasel/38.2.0"
192.168.56.102 - - [03/Sep/2015:00:31:54 -0700] "GET /dvwa/hackable/uploads/phpshell2.php HTTP/1.1" 200 2 "-" "Mozilla/5.0 (X11; Linux x86_64; rv:38.0) Gecko/20100101 Firefox/38.0 Iceweasel/38.2.0"
```

